## Building the Intuition for LoRA

In [1]:
import torch
import numpy as np
_=torch.manual_seed(0)

## Creating a Rank-Deficient matrix W

In [3]:
## rank deficient means basically,the number of independent vectors/columns are lesser than the dimension of the matrix
d,k=10,10

w_rank=2
W=torch.randn(d,w_rank)@torch.randn(w_rank,d)
print(W)


tensor([[ 2.8501, -4.1679, -1.2931, -1.7376, -2.5698, -3.2220, -1.4271, -1.2982,
          0.2702,  1.2163],
        [ 3.2737, -4.7411, -1.4644, -1.9621, -2.9216, -3.6760, -1.6166, -1.4949,
          0.2975,  1.3819],
        [-0.0141, -3.3560, -1.5177, -2.4550, -2.1852, -1.7979, -1.6433,  0.2801,
          0.9375,  1.1010],
        [-0.8365,  0.4910,  0.0490, -0.0243,  0.2776,  0.5523,  0.0609,  0.4404,
          0.1243, -0.1169],
        [-3.9740, -0.6857, -1.1295, -2.3176, -0.6460,  1.0025, -1.1858,  2.3367,
          1.4298,  0.4341],
        [ 0.7376, -0.9989, -0.2987, -0.3915, -0.6132, -0.7910, -0.3304, -0.3424,
          0.0478,  0.2886],
        [-2.2472,  1.8582,  0.3750,  0.3281,  1.0966,  1.7733,  0.4272,  1.1393,
          0.1840, -0.4908],
        [ 0.7821, -0.5984, -0.1087, -0.0790, -0.3502, -0.5912, -0.1251, -0.4004,
         -0.0775,  0.1550],
        [-0.0482, -0.4016, -0.1912, -0.3150, -0.2638, -0.1991, -0.2066,  0.0602,
          0.1267,  0.1342],
        [ 0.6151, -

### Evaluate the rank of the matrix using SVD

In [5]:
W_rank=np.linalg.matrix_rank(W)
print(f"rank of W is:{W_rank}") ## these are the number of linearly independent columns/vectors

rank of W is:2


In [14]:
## SVD on W (W=U*S*V^T)
U,S,V=torch.svd(W)

## for rank-r factorization, we are keeping only first r singular values (and correspoding columns of U and V)
U_r=U[:,:W_rank]
S_r=torch.diag(S[:W_rank])
V_r=V[:,:W_rank].transpose(-1,-2) # Transpose V_r to get the right dimensions

## computing C=U_r * S_r and R=V_r
B=U_r @ S_r
A=V_r

print(f"shape of B: {B.shape}")
print(f"shape of A:{A.shape}")

## These are the low rank representations of W (LORA)--> 10*2 and 2*10


shape of B: torch.Size([10, 2])
shape of A:torch.Size([2, 10])


In [20]:
## now generating random bias and input
bias =torch.randn(d)
x=torch.randn(d)

y=W@x+bias ## broadcasting will automatically take care of dimensions

## computing y'=CRx+b
y_prime=(B@A)@x+bias

print("original y using W:\n",y)
print("\n")
print("y computed using BA:\n",y_prime)


original y using W:
 tensor([-0.0038,  2.8550,  2.2213,  0.3943,  2.1021,  2.0980, -1.8128,  0.1020,
         1.4823,  1.6640])


y computed using BA:
 tensor([-0.0038,  2.8550,  2.2213,  0.3943,  2.1021,  2.0980, -1.8128,  0.1020,
         1.4823,  1.6640])


In [21]:
print("Total number of elements of W:",W.nelement())
print("Total number of element of B and A",B.nelement()+A.nelement())

Total number of elements of W: 100
Total number of element of B and A 40
